In [1]:
import os
import random
import shutil
from math import floor
import cv2
import numpy as np

In [2]:
def move_videos_to_test(src_dir, dest_dir, percentage):
    videos = [f for f in os.listdir(src_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    num_to_move = floor(len(videos) * percentage)
    
    videos_to_move = random.sample(videos, num_to_move)
    
    for video in videos_to_move:
        src_path = os.path.join(src_dir, video)
        dest_path = os.path.join(dest_dir, video)
        shutil.move(src_path, dest_path)
        print(f"Moved {video} to {dest_dir}")

In [3]:
dataset_dir = "Dataset"
train_dataset_dir = os.path.join(dataset_dir, "Train")
deepfake_dir = os.path.join(train_dataset_dir, "DeepFake")
real_dir = os.path.join(train_dataset_dir, "Real")

test_dir = os.path.join(dataset_dir, "Test")
test_deepfake_dir = os.path.join(test_dir, "DeepFake")
test_real_dir = os.path.join(test_dir, "Real")

os.makedirs(test_deepfake_dir, exist_ok=True)
os.makedirs(test_real_dir, exist_ok=True)

move_videos_to_test(deepfake_dir, test_deepfake_dir, 0.20)

move_videos_to_test(real_dir, test_real_dir, 0.10)

print("Video splitting and moving completed.")

Moved id30_id3_0001.mp4 to Dataset\Test\DeepFake
Moved id56_id52_0000.mp4 to Dataset\Test\DeepFake
Moved id51_id53_0006.mp4 to Dataset\Test\DeepFake
Moved id1_id6_0005.mp4 to Dataset\Test\DeepFake
Moved id2_id3_0009.mp4 to Dataset\Test\DeepFake
Moved id56_id54_0007.mp4 to Dataset\Test\DeepFake
Moved id30_id2_0004.mp4 to Dataset\Test\DeepFake
Moved id59_id5_0001.mp4 to Dataset\Test\DeepFake
Moved id50_id55_0007.mp4 to Dataset\Test\DeepFake
Moved id34_id32_0000.mp4 to Dataset\Test\DeepFake
Moved id2_id16_0002.mp4 to Dataset\Test\DeepFake
Moved id31_id2_0005.mp4 to Dataset\Test\DeepFake
Moved id31_id17_0006.mp4 to Dataset\Test\DeepFake
Moved id0_id16_0009.mp4 to Dataset\Test\DeepFake
Moved id20_id38_0006.mp4 to Dataset\Test\DeepFake
Moved id28_id0_0000.mp4 to Dataset\Test\DeepFake
Moved id2_id31_0007.mp4 to Dataset\Test\DeepFake
Moved id33_id32_0003.mp4 to Dataset\Test\DeepFake
Moved id6_id31_0004.mp4 to Dataset\Test\DeepFake
Moved id17_id20_0004.mp4 to Dataset\Test\DeepFake
Moved id0_id2

In [3]:
dataset_dir = "Dataset/Train"
deepfake_dir = os.path.join(dataset_dir, "DeepFake")
real_dir = os.path.join(dataset_dir, "Real")

output_dir = "Dataset/Train/Frames"
deepfake_frames_dir = os.path.join(output_dir, "DeepFake Frames")
real_frames_dir = os.path.join(output_dir, "Real Frames")

os.makedirs(deepfake_frames_dir, exist_ok=True)
os.makedirs(real_frames_dir, exist_ok=True)


In [12]:
def extract_frames_from_video(video_path, output_dir, max_frames):
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(video_path).split('.')[0]
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // max_frames if max_frames < total_frames else 1

    frame_count = 0
    saved_frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or saved_frame_count >= max_frames:
            break

        if frame_count % frame_interval == 0 and saved_frame_count < max_frames:
            frame_filename = f"{video_name}_frame_{saved_frame_count}.jpg"
            frame_output_path = os.path.join(output_dir, frame_filename)
            cv2.imwrite(frame_output_path, frame)
            saved_frame_count += 1

        frame_count += 1

    cap.release()

In [5]:
for video_file in os.listdir(deepfake_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):
        video_path = os.path.join(deepfake_dir, video_file)
        extract_frames_from_video(video_path, deepfake_frames_dir, max_frames=3) 

for video_file in os.listdir(real_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):
        video_path = os.path.join(real_dir, video_file)
        extract_frames_from_video(video_path, real_frames_dir, max_frames=15)  

print("Frame extraction completed.")

Frame extraction completed.


In [6]:
frames_dir = "Dataset/Train/Frames"
deepfake_frames_dir = os.path.join(frames_dir, "DeepFake Frames")
real_frames_dir = os.path.join(frames_dir, "Real Frames")
preprocessed_dir = "Dataset/Train/Preprocessed"

preprocessed_deepfake_dir = os.path.join(preprocessed_dir, "DeepFake")
preprocessed_real_dir = os.path.join(preprocessed_dir, "Real")
os.makedirs(preprocessed_deepfake_dir, exist_ok=True)
os.makedirs(preprocessed_real_dir, exist_ok=True)

In [7]:
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    
    image = cv2.resize(image, target_size)

    image = image.astype(np.float32) / 255.0

    return image

In [8]:
def process_frames(frames_dir, output_dir, label):
    for frame_file in os.listdir(frames_dir):
        if frame_file.endswith(('.jpg', '.jpeg', '.png')):
            frame_path = os.path.join(frames_dir, frame_file)

            preprocessed_image = preprocess_image(frame_path)

            output_path = os.path.join(output_dir, frame_file)  
            cv2.imwrite(output_path, (preprocessed_image * 255).astype(np.uint8))  

In [9]:
process_frames(deepfake_frames_dir, preprocessed_deepfake_dir, label=1)

process_frames(real_frames_dir, preprocessed_real_dir, label=0)

print("Preprocessing completed. Preprocessed training images are saved in:", preprocessed_dir)

Preprocessing completed. Preprocessed training images are saved in: Dataset/Train/Preprocessed


In [10]:
dataset_dir = "Dataset/Test"
deepfake_dir = os.path.join(dataset_dir, "DeepFake")
real_dir = os.path.join(dataset_dir, "Real")

output_dir = "Dataset/Test/Frames"
deepfake_frames_dir = os.path.join(output_dir, "DeepFake Frames")
real_frames_dir = os.path.join(output_dir, "Real Frames")

os.makedirs(deepfake_frames_dir, exist_ok=True)
os.makedirs(real_frames_dir, exist_ok=True)

In [13]:
for video_file in os.listdir(deepfake_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):
        video_path = os.path.join(deepfake_dir, video_file)
        extract_frames_from_video(video_path, deepfake_frames_dir, max_frames=3) 

for video_file in os.listdir(real_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):
        video_path = os.path.join(real_dir, video_file)
        extract_frames_from_video(video_path, real_frames_dir, max_frames=15)  

print("Frame extraction completed.")

Frame extraction completed.


In [15]:
frames_dir = "Dataset/Test/Frames"
deepfake_frames_dir = os.path.join(frames_dir, "DeepFake Frames")
real_frames_dir = os.path.join(frames_dir, "Real Frames")
preprocessed_dir = "Dataset/Test/Preprocessed"

preprocessed_deepfake_dir = os.path.join(preprocessed_dir, "DeepFake")
preprocessed_real_dir = os.path.join(preprocessed_dir, "Real")
os.makedirs(preprocessed_deepfake_dir, exist_ok=True)
os.makedirs(preprocessed_real_dir, exist_ok=True)

In [16]:
process_frames(deepfake_frames_dir, preprocessed_deepfake_dir, label=1)

process_frames(real_frames_dir, preprocessed_real_dir, label=0)

print("Preprocessing completed. Preprocessed test images are saved in:", preprocessed_dir)

Preprocessing completed. Preprocessed test images are saved in: Dataset/Test/Preprocessed
